In [1]:
import os
import pyspark.sql.functions as funcs
from pyspark.sql import Row
import time

import re

THRESHOLD = 0.001

if not os.path.isfile("test.txt"):
    print("please add: " + os.getcwd())
    FileText= os.path.file("test.txt")


In [2]:

%pylab inline
from pyspark import SparkContext, SparkConf

# transfer df to rdd
dfFile = spark.read.load("C:\\Users\\sgoel\\PycharmProjects\\University\\BigData\\TargilTwo\\test.txt",
                     format="csv", sep='\t', inferSchema="true", header="true")
df = dfFile.select("AnonID", "Query").distinct()
queryData = df.rdd.map(list)



Populating the interactive namespace from numpy and matplotlib


In [3]:
print(queryData.take(2))

[[142, 'staple.com'], [142, 'www.newyorklawyersite.com']]


In [4]:
# number of users
usersCount = queryData.groupByKey().count()

In [5]:
print(usersCount)

2


In [6]:
queries = queryData.map(lambda line: line[1])
#  support calculation  
querySupport = queries.map(lambda query: (query, 1)) \
    .reduceByKey(lambda k1, k2: k1 + k2) \
    .map(lambda x: (x[0], x[1] / usersCount)) \
    .filter(lambda x: x[1] > THRESHOLD)

remainQueries = querySupport.map(lambda x:x[0]).collect()


In [7]:
print(remainQueries)

['www.newyorklawyersite.com', 'www.prescriptionfortime.com', 'westchester.gov', 'lottery', 'rentdirect.com', 'staple.com']


In [9]:
#Remove all quries with lower support then TH
def remainAboveTHQueries(queriesList, remainQueriesList):
    return [s for s in queriesList if s in remainQueriesList]

def createPairs(arr):
        pairs = []
        for q1 in range(len(arr)):
                for q2 in range(q1+1,len(arr)):
                        pairs.append((arr[q1],arr[q2]))
        return pairs
# sort for  (b,a)=>(a,b) 
def sortForUnion(arr):
    if(arr[0] <= arr[1]):
        return arr
    return [arr[1],arr[0]]



In [10]:
userQueryTuple = queryData.groupByKey().mapValues(list).filter(lambda t: len(t[1]) > 1)  
userQueryTuple = userQueryTuple.map(lambda t: (t[0], remainAboveTHQueries(t[1], remainQueries)))
userQueryTuple.take(2)

[(142,
  ['staple.com',
   'www.newyorklawyersite.com',
   'rentdirect.com',
   'www.prescriptionfortime.com',
   'westchester.gov'])]

In [11]:
quriesTuple = userQueryTuple.map(lambda t: t[1]).flatMap(lambda arr: createPairs(arr))
quriesTuple.take(30)


[('staple.com', 'www.newyorklawyersite.com'),
 ('staple.com', 'rentdirect.com'),
 ('staple.com', 'www.prescriptionfortime.com'),
 ('staple.com', 'westchester.gov'),
 ('www.newyorklawyersite.com', 'rentdirect.com'),
 ('www.newyorklawyersite.com', 'www.prescriptionfortime.com'),
 ('www.newyorklawyersite.com', 'westchester.gov'),
 ('rentdirect.com', 'www.prescriptionfortime.com'),
 ('rentdirect.com', 'westchester.gov'),
 ('www.prescriptionfortime.com', 'westchester.gov')]

In [12]:
sortedQueriesTuple = quriesTuple.map(lambda t: sortForUnion(list(t))) \
    .map(lambda arr: (arr[0], arr[1]))

sortedQueriesTupleCount = sortedQueriesTuple \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda k1, k2: k1 + k2) \
    .filter(lambda t: t[1] > 1) \
    .map(lambda x: (x[0], x[1] / usersCount))

sortedQueriesTupleCount.take(10)


[]

In [13]:
# todo: need to fix 
cartesianTuple = sortedQueriesTupleCount.cartesian(querySupport)
cartesianTuple.take(3)

In [ ]:
# calculate confidence (XUY/X)
leftConfidence = cartesianTuple \
    .filter(lambda l: l[0][0][0] == l[1][0]) \
    .map(lambda l: (l[0][0][0], l[0][0][1], float(l[0][1]) / l[1][1]))

print(leftConfidence.take(5))

rightConfidence = cartesianTuple \
    .filter(lambda lr: lr[0][0][1] == lr[1][0]) \
    .map(lambda lr: (lr[0][0][1], lr[0][0][0], float(lr[0][1]) / lr[1][1]))
print(rightConfidence.take(5))

rdd_query_conf = sc.union([leftConfidence, rightConfidence])


[('a', 'b', 0.6666666666666666), ('a', 'c', 0.6666666666666666)]
[('b', 'a', 1.0), ('c', 'a', 1.0)]


Py4JJavaError: An error occurred while calling o399.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/yo/test/Untitled Folder/ex02_all_confs already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1096)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairR